In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=qZ4GxOw4U5_AZCc5ab5TH58UCxaD7NFuprTy7ReXyTU&tc=W6cgbYRzZbOPWLJc9gU9_VGZp9u8yG0rVdGgX1SHL3M&cc=6l-oYL_MgrsK-OaNiHNouTZO4T2mZPbWZWWd3nRFf8g

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk6wEZB6T2_2Gg8GBK34GtGVotF9GJmMuT7vtldMc9V4bxmU7Q5x-aE

Successfully saved authorization token.


In [ ]:
# Define a function to calculate NIRv with quality control
def NIRv(image):
  nirv=image.expression(
      'nir *0.0001 * float((nir*0.0001 - red*0.0001)/(nir*0.0001 + red*0.0001)-L)',
      {
        'nir': image.select('Nadir_Reflectance_Band2'),
        'red': image.select('Nadir_Reflectance_Band1'),
        'L': 0.08
      })
  #qa1 = image.select('BRDF_Albedo_Band_Mandatory_Quality_Band1')
  #qa2 = image.select('BRDF_Albedo_Band_Mandatory_Quality_Band2')
  #mask = qa1.bitwiseAnd(2**0).eq(0) and qa2.bitwiseAnd(2**0).eq(0) # Bit 0 is set to 1 for low-quality pixels
  return image.addBands(nirv.rename('NIRv'))

In [ ]:
# Set up a mask to remove low-quality pixels based on QA band
def c2mask(image):
    qa1=image.select('BRDF_Albedo_Band_Mandatory_Quality_Band1').unmask()
    qa2=image.select('BRDF_Albedo_Band_Mandatory_Quality_Band2').unmask()
    nirv=image.updateMask(qa1.lte(1) .And (qa2.lte(1)))
    return nirv.toFloat()

In [ ]:
# Define the general AOI
aoi = ee.Geometry.Polygon(
        [[[-180, -90],
          [-180, 90],
          [180, 90],
          [180, -90],
          [-180, -90]]],
        None,
        False)

In [ ]:
import numpy as np
import time

In [ ]:
# Define the start and end dates for the study period -- LOOP
#note: GEE does not read the end date data
md=['01-01','01-06','01-06','01-11','01-11','01-16','01-16','01-21','01-21','01-26','01-26','01-31',
    '01-31','02-05','02-05','02-10','02-10','02-15','02-15','02-20','02-20','02-25','02-25','03-02',
    '03-02','03-07','03-07','03-12','03-12','03-17','03-17','03-22','03-22','03-27','03-27','04-01',
    '04-01','04-06','04-06','04-11','04-11','04-16','04-16','04-21','04-21','04-26','04-26','05-01',
    '05-01','05-06','05-06','05-11','05-11','05-16','05-16','05-21','05-21','05-26','05-26','05-31',
    '05-31','06-05','06-05','06-10','06-10','06-15','06-15','06-20','06-20','06-25','06-25','06-30',
    '06-30','07-05','07-05','07-10','07-10','07-15','07-15','07-20','07-20','07-25','07-25','07-30',
    '07-30','08-04','08-04','08-09','08-09','08-14','08-14','08-19','08-19','08-24','08-24','08-29',
    '08-29','09-03','09-03','09-08','09-08','09-13','09-13','09-18','09-18','09-23','09-23','09-28',
    '09-28','10-03','10-03','10-08','10-08','10-13','10-13','10-18','10-18','10-23','10-23','10-28',
    '10-28','11-02','11-02','11-07','11-07','11-12','11-12','11-17','11-17','11-22','11-22','11-27',
    '11-27','12-02','12-02','12-07','12-07','12-12','12-12','12-17','12-17','12-22','12-22','12-27',
    '12-27','01-01']

In [ ]:
#test start & end date -- single year
for yr in range(2002,2003):
  for pd in range(73):
    #print(pd)
    td1=md[pd*2]
    td2=md[pd*2+1]
    start_date = str(yr)+'-'+td1
    end_date   = str(yr)+'-'+td2
    #print(start_date,end_date)
    #Bingo

In [ ]:
#cool & then save the array to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
#full readin 2002-2022
for yr in range(2001,2023):
  for pd in range(73):
    print(pd)
    td1=md[pd*2]
    td2=md[pd*2+1]
    if pd<72:
        start_date = str(yr)+'-'+td1
        end_date   = str(yr)+'-'+td2
    if pd==72:
        start_date = str(yr)+'-'+td1
        end_date   = str(yr+1)+'-'+td2
    print(start_date,end_date)

    # Load the MCD43A4 data for the study period and filter by the study area
    mcd43a4 = ee.ImageCollection('MODIS/006/MCD43A4') \
            .filterDate(start_date, end_date) \
            .filterBounds(aoi)

    # Map the function over the image collection
    nirv_collection = mcd43a4.map(NIRv)
    # Map the function over the NIRv collection
    masked_nirv_collection = nirv_collection.map(c2mask).mean()
    projection=masked_nirv_collection.select('NIRv').projection()
    transform1 = projection.getInfo()['transform']
    resampled_image = masked_nirv_collection.reproject(crs=masked_nirv_collection.select('NIRv').projection(), crsTransform=transform1)

    #convert to array
    #start_time=time.time()
    na_glo=np.ones((130,350))*-1
    roi=ee.Geometry.Rectangle([-170.00, -60.00, 180.00, 70.00],proj='EPSG:4326')
    nirv_array3 = np.array(resampled_image.sampleRectangle(region=roi, defaultValue=-1).getInfo()['properties']['NIRv'])
    na_glo=nirv_array3
    
    np.save('/your own directory/'+str(yr)+'/NIRv_1d_qcl_'+str(yr)+'_'+str(pd+1)+'.npy',na_glo)
    #save to google drive############################################################
    #folder_name = 'NIRv_array_real_1deg/'+str(yr)
    #folder_path = '/content/drive/My Drive/' + folder_name
    #if not os.path.exists(folder_path):
        #os.makedirs(folder_path)
    #array_file_path = folder_path + '/NIRV_'+str(yr)+'_'+str(pd+1)+'.npy'
    #np.save(array_file_path, na_glo)